In [1]:
# Celda 0 — Config
from pathlib import Path
import pandas as pd, numpy as np, math, re, os, json

# Raíz de outputs del proyecto
OUT = Path("/home/cesar/proyectos/TFM_SNN/outputs")

# Carpeta donde guardaremos los artefactos de este informe
SUMMARY = OUT / "summary" / "paper_set_accurate_2025-11-06"
SUMMARY.mkdir(parents=True, exist_ok=True)

# -------- Filtros "paper set" --------
PRESET_KEEP       = {"fast"}     # None para no filtrar por preset
ENCODER_KEEP      = {"rate"}         # None para no filtrar por encoder
SEED_KEEP         = None             # None para no filtrar por semilla {42}

# 👉 No filtrar por métodos (incluye composites sin tocarlos)
METHODS_KEEP      = None

ONLY_NEW_RUNNER   = True             # Solo runs con run_row.json (o run_row.csv)
MTIME_FROM        = pd.Timestamp("2025-11-14")  # fecha corte (incluido)

# -------- Comparabilidad "dura" --------
STRICT_CFG        = True             # mismo modelo/T/AMP/batch_size

# -------- Métrica compuesta --------
ALPHA_COMPOSITE   = 0.5              # peso de MAE vs olvido

# -------- Opciones de informe / narrativa --------
IGNORE_NAIVE_IN_REPORTS = False      # True para ocultar naive en tablas/gráficos finales
GROUP_BY_FULL_METHOD    = False      # False → agrupa por 'method_base'; True → por 'method' (separa composites)
RELATIVE_BASELINE       = "ewc"    # "naive", "ewc" o None. Calcula deltas vs este baseline
BASELINE_MATCH_STRICT   = True       # emparejar baseline por (preset,encoder,model,T,amp,batch,seed)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)


In [2]:
# Celda 1 — Utils

def _safe_float(x, default=np.nan):
    try:
        if x is None: return default
        if isinstance(x, (int, float)): return float(x)
        s = str(x).strip()
        if s.lower() in {"nan","none","null",""}: return default
        return float(s)
    except Exception:
        return default

def _read_json(p: Path):
    try:
        if p.exists():
            return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

def _read_csv_df(p: Path):
    try:
        if p.exists():
            return pd.read_csv(p)
    except Exception:
        pass
    return None

def _abs_run_dir(rel: str|Path) -> Path:
    rel = str(rel)
    return (OUT / rel) if not rel.startswith(str(OUT)) else Path(rel)

def run_mtime(rel: str|Path) -> float:
    rd = _abs_run_dir(rel)
    mt = 0.0
    for root, _, files in os.walk(rd):
        for f in files:
            try:
                mt = max(mt, (Path(root)/f).stat().st_mtime)
            except Exception:
                pass
    return mt

def canonical_method(m: str) -> str:
    if not m: return "none"
    m = m.lower()
    if m.startswith("ewc"):        return "ewc"
    if m.startswith("as-snn"):     return "as-snn"
    if m.startswith("sa-snn"):     return "sa-snn"
    if m.startswith("sca-snn"):    return "sca-snn"
    if m.startswith("rehearsal"):  return "rehearsal"
    if m.startswith("naive"):      return "naive"
    return m

def _read_forgetting(run_dir: Path) -> dict:
    """
    Lee primero forgetting_summary.json (preferente).
    Si no existe, lee forgetting.json (anidado por tareas) y calcula la media.
    Devuelve claves normalizadas: circuito1_forget_abs/rel, circuito2_forget_abs/rel, avg_forget_rel.
    """
    def _nanmean2(a, b):
        vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
        return float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan

    # 1) summary (preferido)
    summ = _read_json(run_dir / "forgetting_summary.json")
    if isinstance(summ, dict):
        c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
        c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
        c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
        c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
        avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
        if math.isnan(avg_rel):
            avg_rel = _nanmean2(c1_rel, c2_rel)
        return {
            "circuito1_forget_abs": c1_abs,
            "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs,
            "circuito2_forget_rel": c2_rel,
            "avg_forget_rel":       avg_rel,
        }

    # 2) forgetting.json (anidado por tareas) o plano
    js = _read_json(run_dir / "forgetting.json") or {}
    if not isinstance(js, dict):
        return {}

    def pick(d, *keys):
        for k in keys:
            if k in d: return d[k]
        return None

    c1_abs = _safe_float(pick(js, "circuito1_forget_abs","task_1_circuito1_forget_abs","c1_forget_abs","task_1_forget_abs"))
    c1_rel = _safe_float(pick(js, "circuito1_forget_rel","task_1_circuito1_forget_rel","c1_forget_rel","task_1_forget_rel"))
    c2_abs = _safe_float(pick(js, "circuito2_forget_abs","task_2_circuito2_forget_abs","c2_forget_abs","task_2_forget_abs"))
    c2_rel = _safe_float(pick(js, "circuito2_forget_rel","task_2_circuito2_forget_rel","c2_forget_rel","task_2_forget_rel"))
    avg_rel = _safe_float(pick(js, "avg_forget_rel","avg_forgetting_rel","mean_forget_rel","forget_rel_avg"))

    # Si sigue faltando, leer anidado por tarea
    if ("circuito1" in js) and isinstance(js["circuito1"], dict):
        if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
        if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
    if ("circuito2" in js) and isinstance(js["circuito2"], dict):
        if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
        if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))

    if math.isnan(avg_rel):
        avg_rel = _nanmean2(c1_rel, c2_rel)

    return {
        "circuito1_forget_abs": c1_abs,
        "circuito1_forget_rel": c1_rel,
        "circuito2_forget_abs": c2_abs,
        "circuito2_forget_rel": c2_rel,
        "avg_forget_rel":       avg_rel,
    }


In [3]:
# Celda 2 — Reconstrucción 100% desde ficheros (robusta a formatos)

import os, re, json, math
from pathlib import Path
import numpy as np
import pandas as pd

# ——— Fallback por si _read_forgetting no quedó definido en la Celda 1 ———
try:
    _read_forgetting
except NameError:
    def _read_forgetting(run_dir: Path):
        def _nanmean2(a, b):
            vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
            return (float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan)
        summ = _read_json(run_dir / "forgetting_summary.json")
        if isinstance(summ, dict):
            c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
            c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
            c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
            c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
            avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
            if math.isnan(avg_rel): avg_rel = _nanmean2(c1_rel, c2_rel)
            return {
                "circuito1_forget_abs": c1_abs, "circuito1_forget_rel": c1_rel,
                "circuito2_forget_abs": c2_abs, "circuito2_forget_rel": c2_rel,
                "avg_forget_rel": avg_rel,
            }
        js = _read_json(run_dir / "forgetting.json") or {}
        if not isinstance(js, dict): return {}
        def pick(d, *keys):
            for k in keys:
                if k in d: return d[k]
            return None
        c1_abs = _safe_float(pick(js, "circuito1_forget_abs","task_1_circuito1_forget_abs","c1_forget_abs","task_1_forget_abs"))
        c1_rel = _safe_float(pick(js, "circuito1_forget_rel","task_1_circuito1_forget_rel","c1_forget_rel","task_1_forget_rel"))
        c2_abs = _safe_float(pick(js, "circuito2_forget_abs","task_2_circuito2_forget_abs","c2_forget_abs","task_2_forget_abs"))
        c2_rel = _safe_float(pick(js, "circuito2_forget_rel","task_2_circuito2_forget_rel","c2_forget_rel","task_2_forget_rel"))
        avg_rel = _safe_float(pick(js, "avg_forget_rel","avg_forgetting_rel","mean_forget_rel","forget_rel_avg"))
        if ("circuito1" in js) and isinstance(js["circuito1"], dict):
            if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
            if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
        if ("circuito2" in js) and isinstance(js["circuito2"], dict):
            if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
            if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))
        if math.isnan(avg_rel):
            vals = [x for x in [c1_rel, c2_rel] if not math.isnan(_safe_float(x))]
            avg_rel = float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan
        return {
            "circuito1_forget_abs": c1_abs, "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs, "circuito2_forget_rel": c2_rel,
            "avg_forget_rel": avg_rel,
        }

# ——— Helpers internos ———

def _parse_basic_meta(run_dir: Path) -> dict:
    """Extrae preset, method, encoder, model, seed, T, amp, batch_size desde los artefactos.
       Prioridad: run_row.json → manifest de tarea → heurística de nombre de carpeta.
    """
    jrow = _read_json(run_dir / "run_row.json") or {}

    def gj(*ks, default=None):
        obj = jrow
        for k in ks:
            if not isinstance(obj, dict) or (k not in obj):
                return default
            obj = obj[k]
        return obj

    # Campos directos de run_row.json (top-level o dentro de meta/data/training)
    preset   = jrow.get("preset") or gj("meta","preset")
    method   = jrow.get("method") or gj("meta","method")
    encoder  = jrow.get("encoder") or gj("meta","encoder")
    model    = jrow.get("model")   or jrow.get("model_name") or gj("meta","model") or gj("meta","model_name")
    seed     = jrow.get("seed")    or gj("meta","seed")
    T        = jrow.get("T")       or gj("data","T") or gj("meta","data","T") or gj("meta","T")
    amp      = jrow.get("amp")     or gj("training","amp") or gj("meta","amp")
    batch_sz = jrow.get("batch_size") or gj("meta","batch_size")

    # Manifest de la primera tarea (por si faltan cosas)
    man1 = _read_json(run_dir / "task_1_circuito1" / "manifest.json")
    if isinstance(man1, dict):
        meta1 = man1.get("meta", {}) if isinstance(man1.get("meta", {}), dict) else {}
        if not model:        model     = meta1.get("model") or man1.get("model") or man1.get("model_name")
        if batch_sz is None: batch_sz  = meta1.get("batch_size", batch_sz)
        # Claves típicas en tu formato:
        if T is None:        T         = meta1.get("T", T)        # T vive en meta
        if amp is None:      amp       = meta1.get("amp", amp)    # amp vive en meta
        if not encoder:      encoder   = meta1.get("encoder", encoder)
        if not preset:       preset    = meta1.get("preset", preset)
        if not method:       method    = meta1.get("method", method)

    # Heurística de nombre de carpeta si faltan preset/method/encoder
    if not preset or not method or not encoder:
        name = run_dir.name
        m = re.match(r"continual_([^_]+)_([^_].*?)_(rate|latency|raw|image).*", name)
        if m:
            preset  = preset  or m.group(1)
            method  = method  or m.group(2)
            encoder = encoder or m.group(3)

    # Tipados/normalizaciones
    seed    = _safe_float(seed)
    T       = _safe_float(T)
    if isinstance(amp, str):
        amp = amp.strip().lower() in {"true","1","yes","y"}
    elif isinstance(amp, (int, float)):
        amp = bool(amp)
    elif amp is not None and not isinstance(amp, bool):
        amp = None
    batch_sz = _safe_float(batch_sz)

    return dict(
        preset=preset, method=method, encoder=encoder, model=model,
        seed=seed, T=T, amp=amp, batch_size=batch_sz
    )

def _read_per_task_perf(run_dir: Path) -> dict:
    """Devuelve dict por tarea con {'best_mae','final_mae'} normalizadas.
       Soporta per_task_perf.json (lista/dict) y per_task_perf.csv.
       Normaliza nombres de tarea (lower) y fusiona JSON+CSV para rellenar huecos.
    """
    def _norm_row_dictlike(d):
        tname = (str(d.get("task_name") or d.get("task") or d.get("name") or "")).strip().lower()
        # añadimos 'test_mae' como candidato de 'best'
        best_candidates  = ["best_mae","val_best_mae","best","mae_best","min_mae","test_mae"]
        final_candidates = ["final_mae","val_final_mae","val_last_mae","last_mae","mae_last","mae_final"]
        best  = next((d.get(k) for k in best_candidates  if k in d), None)
        final = next((d.get(k) for k in final_candidates if k in d), None)
        return tname, {"best_mae": _safe_float(best), "final_mae": _safe_float(final)}

    out = {}

    # 1) JSON
    js = _read_json(run_dir / "per_task_perf.json")
    if js is not None:
        if isinstance(js, list):
            for row in js:
                if isinstance(row, dict):
                    t, val = _norm_row_dictlike(row)
                    if t:
                        out[t] = val
        elif isinstance(js, dict):
            if all(isinstance(v, dict) for v in js.values()):
                for k, v in js.items():
                    t, val = _norm_row_dictlike({"task_name": k, **v})
                    if t:
                        out[t] = val
            else:
                try:
                    df = pd.DataFrame(js)
                    for _, row in df.iterrows():
                        t, val = _norm_row_dictlike(row.to_dict())
                        if t:
                            out[t] = val
                except Exception:
                    pass

    # 2) CSV (fusiona y rellena NaNs si los hay)
    df = _read_csv_df(run_dir / "per_task_perf.csv")
    if df is not None and not df.empty:
        for _, row in df.iterrows():
            tn = (str(row.get("task_name") or row.get("task") or row.get("name") or "")).strip().lower()
            if not tn:
                continue
            cur = out.get(tn, {"best_mae": np.nan, "final_mae": np.nan})
            if math.isnan(_safe_float(cur.get("best_mae"))):
                cur["best_mae"] = _safe_float(row.get("val_best_mae", row.get("best_mae", row.get("test_mae"))))
            if math.isnan(_safe_float(cur.get("final_mae"))):
                cur["final_mae"] = _safe_float(row.get("val_last_mae", row.get("val_final_mae", row.get("final_mae"))))
            out[tn] = cur

    return out

def _read_efficiency(run_dir: Path):
    """Lee emisiones/tiempo desde efficiency_summary.json; si falta, intenta emissions.csv y run_row.json."""
    j = _read_json(run_dir / "efficiency_summary.json") or {}
    emissions = _safe_float(j.get("emissions_kg"), default=np.nan)
    elapsed   = _safe_float(j.get("elapsed_sec"),   default=np.nan)

    if math.isnan(emissions):
        df = _read_csv_df(run_dir / "emissions.csv")
        if df is not None:
            col = "co2e_kg" if "co2e_kg" in df.columns else ("emissions_kg" if "emissions_kg" in df.columns else None)
            if col:
                emissions = float(df[col].sum())

    if math.isnan(elapsed) or math.isnan(emissions):
        rj = _read_json(run_dir / "run_row.json") or {}
        if math.isnan(elapsed):
            elapsed = _safe_float(rj.get("elapsed_sec"), default=elapsed)
        if math.isnan(emissions):
            emissions = _safe_float(rj.get("emissions_kg"), default=emissions)

    return emissions, elapsed

def _read_eval_matrix(run_dir: Path) -> pd.DataFrame | None:
    """Carga eval_matrix (csv preferente) o reconstruye desde json {'tasks':[], 'mae_matrix':[[]]}."""
    p_csv = run_dir / "eval_matrix.csv"
    if p_csv.exists():
        try:
            return pd.read_csv(p_csv)
        except Exception:
            pass

    js = _read_json(run_dir / "eval_matrix.json")
    if isinstance(js, dict) and ("tasks" in js) and ("mae_matrix" in js):
        tasks = list(js.get("tasks") or [])
        mat   = js.get("mae_matrix") or []
        # Intentamos formato de columnas como en CSV: 'task', 'after_circuito1', 'after_circuito2',...
        try:
            mat = np.array(mat, dtype=float)
            cols = ["task"] + [f"after_{t}" for t in tasks]
            data = {"task": tasks}
            for j, cname in enumerate(cols[1:]):
                colvals = [row[j] if j < len(row) else np.nan for row in mat]
                data[cname] = colvals
            return pd.DataFrame(data)
        except Exception:
            # fallback genérico
            try:
                return pd.DataFrame(js)
            except Exception:
                return None
    return None

def _compute_best_final_from_eval(eval_df: pd.DataFrame, task_token: str):
    """Obtiene BEST/FINAL mirando primero por FILAS (col 'task') y,
    si no existe, hace fallback a columnas que contengan el token."""
    if eval_df is None or eval_df.empty:
        return (np.nan, np.nan)

    # 1) Preferir formato fila: 'task' == task_token
    tcol = None
    for c in eval_df.columns:
        if str(c).strip().lower() == "task":
            tcol = c
            break
    if tcol is not None:
        mask = eval_df[tcol].astype(str).str.lower() == str(task_token).lower()
        if mask.any():
            row = eval_df.loc[mask].iloc[0]
            data_cols = [c for c in eval_df.columns if c != tcol]

            vals = pd.to_numeric(row[data_cols], errors="coerce").values.astype(float)
            finite = np.isfinite(vals)
            best = float(np.nanmin(vals)) if finite.any() else np.nan

            # FINAL = valor en la última columna temporal (p.ej., after_última_tarea)
            final = _safe_float(row[data_cols[-1]])
            return (best, final)

    # 2) Fallback: columnas que contengan el token (formatos antiguos)
    cols = [c for c in eval_df.columns if str(task_token).lower() in str(c).lower()]
    if cols:
        dfc = eval_df[cols].apply(pd.to_numeric, errors="coerce")
        vals = dfc.values.astype(float)
        finite = np.isfinite(vals)
        best = float(np.nanmin(vals)) if finite.any() else np.nan
        final = _safe_float(dfc.iloc[-1, -1])
        return (best, final)

    return (np.nan, np.nan)


def _compute_forgetting_from_eval_matrix(eval_df: pd.DataFrame, per_task: dict):
    """Olvido para c1 tras aprender c2. C2 no olvida (0)."""
    out = {}
    t1_keys = [k for k in (per_task or {}).keys() if "circuito1" in str(k).lower()]
    best_t1 = None
    if t1_keys:
        best_t1 = _safe_float((per_task.get(t1_keys[0]) or {}).get("best_mae"))
    if best_t1 is None or math.isnan(best_t1):
        best_t1, _ = _compute_best_final_from_eval(eval_df, "circuito1")

    _, final_t1_after_last = _compute_best_final_from_eval(eval_df, "circuito1")
    if best_t1 is None or math.isnan(best_t1) or final_t1_after_last is None or math.isnan(final_t1_after_last):
        return out

    forget_abs = max(0.0, final_t1_after_last - best_t1)
    forget_rel = forget_abs / max(1e-9, best_t1)
    out["circuito1_forget_abs"] = forget_abs
    out["circuito1_forget_rel"] = forget_rel
    out["circuito2_forget_abs"] = 0.0
    out["circuito2_forget_rel"] = 0.0
    out["avg_forget_rel"] = forget_rel
    return out

def _read_continual_results(run_dir: Path) -> dict:
    """Fallback final para MAEs desde continual_results.json."""
    j = _read_json(run_dir / "continual_results.json")
    if not isinstance(j, dict):
        return {}
    c1 = j.get("circuito1", {}) or {}
    c2 = j.get("circuito2", {}) or {}
    return {
        "c1_best":  _safe_float(c1.get("test_mae")),
        "c1_final": _safe_float(c1.get("after_circuito2_mae")),
        "c2_best":  _safe_float(c2.get("test_mae")),
        "c2_final": _safe_float(c2.get("test_mae")),
    }

def build_results_table_from_disk(base_out: Path) -> pd.DataFrame:
    rows = []
    run_dirs = [p for p in base_out.glob("continual_*") if p.is_dir()]
    print(f"[INFO] Escaneando {len(run_dirs)} runs en {base_out}")

    for rd in run_dirs:
        meta      = _parse_basic_meta(rd)
        per_task  = _read_per_task_perf(rd)
        eff_kg, elapsed = _read_efficiency(rd)
        forget_js = _read_forgetting(rd) or {}
        eval_df   = _read_eval_matrix(rd)

        # MAEs por tarea con fallback a eval_matrix y, si hace falta, continual_results.json
        c1_best = c1_final = c2_best = c2_final = np.nan

        t1_keys = [k for k in per_task.keys() if "circuito1" in str(k).lower()]
        if t1_keys:
            c1_best  = _safe_float(per_task[t1_keys[0]].get("best_mae"),  default=np.nan)
            c1_final = _safe_float(per_task[t1_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c1_best) or math.isnan(c1_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito1")
            if math.isnan(c1_best):  c1_best  = b
            if math.isnan(c1_final): c1_final = f

        t2_keys = [k for k in per_task.keys() if "circuito2" in str(k).lower()]
        if t2_keys:
            c2_best  = _safe_float(per_task[t2_keys[0]].get("best_mae"),  default=np.nan)
            c2_final = _safe_float(per_task[t2_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c2_best) or math.isnan(c2_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito2")
            if math.isnan(c2_best):  c2_best  = b
            if math.isnan(c2_final): c2_final = f

        # Fallback definitivo: continual_results.json
        if any(math.isnan(x) for x in [c1_best, c1_final, c2_best, c2_final]):
            cr = _read_continual_results(rd)
            if math.isnan(c1_best):  c1_best  = _safe_float(cr.get("c1_best"),  default=c1_best)
            if math.isnan(c1_final): c1_final = _safe_float(cr.get("c1_final"), default=c1_final)
            if math.isnan(c2_best):  c2_best  = _safe_float(cr.get("c2_best"),  default=c2_best)
            if math.isnan(c2_final): c2_final = _safe_float(cr.get("c2_final"), default=c2_final)

        # Olvido (summary/json) o cálculo desde eval_matrix
        f_c1_abs = _safe_float(forget_js.get("circuito1_forget_abs"))
        f_c1_rel = _safe_float(forget_js.get("circuito1_forget_rel"))
        f_c2_abs = _safe_float(forget_js.get("circuito2_forget_abs"))
        f_c2_rel = _safe_float(forget_js.get("circuito2_forget_rel"))
        avg_f_rel = _safe_float(forget_js.get("avg_forget_rel"))
        if all(math.isnan(x) for x in [f_c1_abs, f_c1_rel, f_c2_abs, f_c2_rel, avg_f_rel]):
            comp = _compute_forgetting_from_eval_matrix(eval_df, per_task)
            if comp:
                f_c1_abs = comp.get("circuito1_forget_abs", f_c1_abs)
                f_c1_rel = comp.get("circuito1_forget_rel", f_c1_rel)
                f_c2_abs = comp.get("circuito2_forget_abs", f_c2_abs)
                f_c2_rel = comp.get("circuito2_forget_rel", f_c2_rel)
                avg_f_rel = comp.get("avg_forget_rel", avg_f_rel)

        row = dict(
            run_dir=str(rd.relative_to(base_out)),
            preset=meta["preset"],
            method=meta["method"],
            encoder=meta["encoder"],
            model=meta["model"],
            seed=meta["seed"],
            T=meta["T"],
            batch_size=meta["batch_size"],
            amp=meta["amp"],
            emissions_kg=eff_kg,
            elapsed_sec=elapsed,
            circuito1_best_mae=c1_best,
            circuito1_final_mae=c1_final,
            circuito2_best_mae=c2_best,
            circuito2_final_mae=c2_final,
            circuito1_forget_abs=f_c1_abs,
            circuito1_forget_rel=f_c1_rel,
            circuito2_forget_abs=f_c2_abs,
            circuito2_forget_rel=f_c2_rel,
            avg_forget_rel=avg_f_rel,
        )
        rows.append(row)

    df = pd.DataFrame(rows)

    # Flags extra
    df["is_new_runner"] = df["run_dir"].apply(lambda rd: (_abs_run_dir(rd) / "run_row.json").exists() or (_abs_run_dir(rd) / "run_row.csv").exists())
    df["mtime"] = df["run_dir"].apply(run_mtime)
    df["mtime_dt"] = pd.to_datetime(df["mtime"], unit="s")
    df["method_base"] = df["method"].astype(str).apply(canonical_method)

    # Tipado numérico: EXCLUYE 'amp' (lo normalizamos como boolean más abajo)
    numeric_cols = [
        "seed","T","batch_size","emissions_kg","elapsed_sec",
        "circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae",
        "circuito1_forget_abs","circuito1_forget_rel","circuito2_forget_abs","circuito2_forget_rel","avg_forget_rel"
    ]
    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # 'amp' como boolean (permite NA)
    if "amp" in df.columns:
        if df["amp"].dtype == bool:
            df["amp"] = df["amp"].astype("boolean")
        else:
            df["amp"] = df["amp"].map(
                lambda v: bool(v) if isinstance(v, (bool,int,float))
                else (str(v).strip().lower() in {"true","1","yes","y"} if isinstance(v, str) else pd.NA)
            ).astype("boolean")

    out_csv = SUMMARY / "results_table_fromdisk.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] results_table_fromdisk → {out_csv} | filas:", len(df))
    return df


In [4]:
# Celda 3 — Construcción + diagnóstico NaNs

df_all = build_results_table_from_disk(OUT)
display(df_all.head(3))

nan_cols = ["circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae","avg_forget_rel"]
print("[DEBUG] NaNs globales:", {c:int(df_all[c].isna().sum()) for c in nan_cols if c in df_all.columns})


[INFO] Escaneando 241 runs en /home/cesar/proyectos/TFM_SNN/outputs
[OK] results_table_fromdisk → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/results_table_fromdisk.csv | filas: 241


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base
0,continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_f...,fast,as-snn_gr_0.6_lam_0.36_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005630,771.873337,0.189499,0.262805,0.190483,0.190483,0.073306,0.386843,0.0,0.0,0.193422,True,1.763134e+09,2025-11-14 15:27:35.800247431,as-snn
1,continual_fast_ewc_lam_5e+07_fast_ewc_l5e7_fb3...,fast,ewc_lam_5e+07,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.008511,1921.457078,0.140196,0.231559,0.199714,0.199714,0.091363,0.651682,0.0,0.0,0.325841,True,1.762962e+09,2025-11-12 15:38:42.516785860,ewc
2,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,fast,as-snn_gr_0.6_lam_0.4_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004914,770.022141,0.127710,0.175707,0.165940,0.165940,0.047996,0.375820,0.0,0.0,0.187910,True,1.763093e+09,2025-11-14 04:09:08.899888992,as-snn


[DEBUG] NaNs globales: {'circuito1_best_mae': 9, 'circuito1_final_mae': 9, 'circuito2_best_mae': 9, 'circuito2_final_mae': 9, 'avg_forget_rel': 9}


In [5]:
# Celda 4 — Selección y comparabilidad

def _filter_paperset(df: pd.DataFrame, require_both_tasks: bool = True) -> pd.DataFrame:
    m = df.copy()
    if PRESET_KEEP:
        m = m[m["preset"].isin(PRESET_KEEP)]
    if ENCODER_KEEP:
        m = m[m["encoder"].isin(ENCODER_KEEP)]
    if SEED_KEEP:
        m = m[m["seed"].isin(SEED_KEEP)]
    if METHODS_KEEP:
        m = m[m["method_base"].isin(METHODS_KEEP)]
    if ONLY_NEW_RUNNER:
        m = m[m["is_new_runner"] == True]
    if MTIME_FROM is not None:
        m = m[m["mtime_dt"] >= MTIME_FROM]

    # Runs con ambas tareas evaluadas (mejor y final). Evita arrastrar runs truncados.
    if require_both_tasks:
        must_cols = ["circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae"]
        for c in must_cols:
            if c in m.columns:
                m = m[~m[c].isna()]

    print(f"[DEBUG] filtros duros → inicio → {len(df)} runs | post → {len(m)} runs")
    return m

def _comparability_slice(df: pd.DataFrame, strict: bool = True):
    if df.empty:
        return df, {"estrategia":"empty", "kept":0, "dropped":0}
    # Siempre mismo modelo y mismo T
    modes_model = df["model"].dropna().unique().tolist()
    modes_T     = df["T"].dropna().unique().tolist()
    if len(modes_model) > 1:
        top_model = df["model"].value_counts().idxmax()
        df = df[df["model"] == top_model]
    if len(modes_T) > 1:
        top_T = df["T"].value_counts().idxmax()
        df = df[df["T"] == top_T]

    kept_before = len(df)
    if strict:
        # AMP mayoritario (si hay), manteniendo NaN
        if df["amp"].notna().any():
            top_amp = df["amp"].value_counts(dropna=True).idxmax()
            df = df[(df["amp"].isna()) | (df["amp"] == top_amp)]
        # batch_size mayoritario (si hay), manteniendo NaN
        if df["batch_size"].notna().any():
            top_bs = df["batch_size"].value_counts(dropna=True).idxmax()
            df = df[(df["batch_size"].isna()) | (df["batch_size"] == top_bs)]
        strategy = "strict:model+T(+amp+batch)"
    else:
        strategy = "relaxed:model+T"

    kept_after = len(df)
    return df, {"estrategia": strategy, "kept": kept_after, "dropped": kept_before - kept_after}

df_sel0 = _filter_paperset(df_all, require_both_tasks=True)
df_sel, stats = _comparability_slice(df_sel0, strict=STRICT_CFG)
if df_sel.empty and STRICT_CFG:
    print("[WARN] Comparabilidad dejó 0 runs. Relajando AMP y batch_size…")
    df_sel, stats = _comparability_slice(df_sel0, strict=False)

# Copia para informes (se puede filtrar naive aquí si se desea solo de cara a tablas/plots)
df_sel["batch_size_filled"] = df_sel["batch_size"].copy()
df_report = df_sel.copy()
if IGNORE_NAIVE_IN_REPORTS and "method_base" in df_report.columns:
    df_report = df_report[df_report["method_base"] != "naive"]

display(df_sel.head(10))
print(f"[OK] Selección final: {len(df_sel)} runs | estrategia: {stats}")
out_csv = SUMMARY / "selection_table.csv"
df_sel.to_csv(out_csv, index=False)
print("[OK] Selección →", out_csv)


[DEBUG] filtros duros → inicio → 241 runs | post → 76 runs


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base,batch_size_filled
0,continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_f...,fast,as-snn_gr_0.6_lam_0.36_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005630,771.873337,0.189499,0.262805,0.190483,0.190483,0.073306,0.386843,0.0,0.0,0.193422,True,1.763134e+09,2025-11-14 15:27:35.800247431,as-snn,320.0
2,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,fast,as-snn_gr_0.6_lam_0.4_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.004914,770.022141,0.127710,0.175707,0.165940,0.165940,0.047996,0.375820,0.0,0.0,0.187910,True,1.763093e+09,2025-11-14 04:09:08.899888992,as-snn,320.0
12,continual_fast_sca-snn_bins32_beta0.8_bias0.1_...,fast,sca-snn_bins32_beta0.8_bias0.1_temp0.1_ab12_flat1,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005089,826.357135,0.134410,0.206274,0.167309,0.167309,0.071864,0.534661,0.0,0.0,0.267330,True,1.763103e+09,2025-11-14 06:55:57.203609467,sca-snn,320.0
14,continual_fast_sca-snn_bins32_beta0.8_bias0.1_...,fast,sca-snn_bins32_beta0.8_bias0.1_temp0.12_ab12_f...,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005131,829.244563,0.142382,0.208985,0.172110,0.172110,0.066602,0.467773,0.0,0.0,0.233886,True,1.763080e+09,2025-11-14 00:33:49.389898777,sca-snn,320.0
16,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,fast,as-snn_gr_0.6_lam_0.4_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005178,882.426415,0.162676,0.201612,0.181978,0.181978,0.038935,0.239342,0.0,0.0,0.119671,True,1.763114e+09,2025-11-14 09:58:09.170902491,as-snn,320.0
19,continual_fast_ewc_lam_6e+08_fast3_ewc_l60e8_f...,fast,ewc_lam_6e+08,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.008168,1342.165898,0.143924,0.237559,0.208045,0.208045,0.093635,0.650586,0.0,0.0,0.325293,True,1.763142e+09,2025-11-14 17:45:09.592230797,ewc,320.0
21,continual_fast_sa-snn_fast3_sa_k0p08_tau24_vt1...,fast,sa-snn,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005046,816.013944,0.130911,0.174971,0.169596,0.169596,0.044060,0.336562,0.0,0.0,0.168281,True,1.763099e+09,2025-11-14 05:40:59.276646376,sa-snn,320.0
25,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,fast,sca-snn_bins32_beta0.8_bias0.12_temp0.12_ab16_...,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005115,831.363631,0.140053,0.201127,0.184444,0.184444,0.061075,0.436083,0.0,0.0,0.218041,True,1.763105e+09,2025-11-14 07:23:45.278599977,sca-snn,320.0
27,continual_fast_ewc_lam_4e+08_fast3_ewc_l35e8_f...,fast,ewc_lam_4e+08,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.009003,1534.300150,0.143851,0.227676,0.198425,0.198425,0.083825,0.582721,0.0,0.0,0.291360,True,1.763144e+09,2025-11-14 18:10:46.369233131,ewc,320.0
35,continual_fast_sa-snn_fast3_sa_k0p12_tau24_vt1...,fast,sa-snn,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,0.005826,814.359869,0.136611,0.213242,0.168165,0.168165,0.076632,0.560951,0.0,0.0,0.280475,True,1.763147e+09,2025-11-14 18:57:02.178255320,sa-snn,320.0


[OK] Selección final: 62 runs | estrategia: {'estrategia': 'strict:model+T(+amp+batch)', 'kept': 62, 'dropped': 0}
[OK] Selección → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/selection_table.csv


In [6]:
# Celda 5 — Ranking compuesto + winners por método (versión robusta)

def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Crea alias entre c2_final_mae y circuito2_final_mae si falta alguno."""
    df = df.copy()
    has_c2    = "c2_final_mae" in df.columns
    has_circ2 = "circuito2_final_mae" in df.columns
    if has_c2 and not has_circ2:
        df["circuito2_final_mae"] = df["c2_final_mae"]
    elif has_circ2 and not has_c2:
        df["c2_final_mae"] = df["circuito2_final_mae"]
    return df

def _minmax_norm(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    if s.size == 0:
        return s
    vals = s.values.astype(float)
    finite = np.isfinite(vals)
    if finite.sum() == 0:
        return pd.Series(np.full(len(s), np.nan), index=s.index)
    lo, hi = float(np.nanmin(vals)), float(np.nanmax(vals))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        out = np.zeros(len(s), dtype=float)
        out[~finite] = np.nan
        return pd.Series(out, index=s.index)
    return (s - lo) / (hi - lo)

# 1) Copia y compatibilidad de columnas
df_rank = _ensure_mae_columns(df_report)

# Columnas clave
MAE_COL  = "circuito2_final_mae"
FORG_COL = "avg_forget_rel"

# 2) Si está vacío, escribe CSV vacío y sal con aviso
if df_rank.empty:
    winners = df_rank.copy()
    out_csv = SUMMARY / "winners_per_method.csv"
    winners.to_csv(out_csv, index=False)
    print("[WARN] df_rank está vacío tras filtros. Winners vacío →", out_csv)
    display(winners)  # DF vacío
else:
    mae_norm  = _minmax_norm(df_rank[MAE_COL]  if MAE_COL  in df_rank.columns else pd.Series([], dtype=float))
    forg_norm = _minmax_norm(df_rank[FORG_COL] if FORG_COL in df_rank.columns else pd.Series([], dtype=float))

    score_comp = ALPHA_COMPOSITE * mae_norm + (1 - ALPHA_COMPOSITE) * forg_norm
    score_eff  = score_comp.copy()
    if len(score_eff) != 0:
        score_eff[forg_norm.isna()] = mae_norm[forg_norm.isna()]  # fallback

    df_rank["score_comp"] = score_comp
    df_rank["score_eff"]  = score_eff

    # Agrupación: por método base o por método completo (separa composites)
    group_col = "method" if GROUP_BY_FULL_METHOD else "method_base"
    if group_col not in df_rank.columns:
        df_rank[group_col] = df_rank["method"] if "method" in df_rank.columns else "unknown"

    # Winners
    if df_rank["score_eff"].notna().any():
        winners = (
            df_rank
            .sort_values([group_col, "score_eff", MAE_COL], ascending=[True, True, True])
            .groupby(group_col, as_index=False)
            .head(1)
            .sort_values("score_eff", ascending=True)
        )
    else:
        winners = (
            df_rank
            .sort_values([group_col, MAE_COL], ascending=[True, True])
            .groupby(group_col, as_index=False)
            .head(1)
        )

    # Export
    out_csv = SUMMARY / f"winners_per_{'fullmethod' if GROUP_BY_FULL_METHOD else 'methodbase'}.csv"
    winners.to_csv(out_csv, index=False)

    n_rows = len(df_rank)
    n_groups = df_rank[group_col].nunique()
    print(f"[OK] Winners → {out_csv} | candidatos={n_rows} | grupos={n_groups}")
    cols_show = [c for c in ["run_dir", group_col, "seed", MAE_COL, FORG_COL, "emissions_kg", "score_eff"] if c in winners.columns]
    display(winners[cols_show] if cols_show else winners)

# Mantén "winners" en el entorno por si otras celdas lo usan


[OK] Winners → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/winners_per_methodbase.csv | candidatos=62 | grupos=4


,run_dir,method_base,seed,circuito2_final_mae,avg_forget_rel,emissions_kg,score_eff
210,continual_fast_as-snn_gr_0.55_lam_0.4_att_f6_f...,as-snn,42.0,0.163275,0.095513,0.005170,0.107657
108,continual_fast_sca-snn_bins32_beta0.8_bias0.08...,sca-snn,42.0,0.160026,0.120142,0.004915,0.114184
65,continual_fast_sa-snn_fast3_sa_k0p10_tau28_vt1...,sa-snn,42.0,0.157522,0.157920,0.005827,0.135323
218,continual_fast_ewc_lam_4e+08_fast3_ewc_l35e8_f...,ewc,42.0,0.204162,0.065061,0.007266,0.265020


In [7]:
# Celda 5.1 — Deltas relativos vs baseline (para narrativa TFM)

def _row_key_for_match(r):
    """Clave de emparejamiento para baseline estricto."""
    return (
        r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
        r.get("amp"), r.get("batch_size"), r.get("seed")
    )

def compute_relative_to_baseline(df_in: pd.DataFrame, baseline="naive", strict=True):
    if not baseline:
        return None
    dfb = df_in.copy()
    dfb["__key__"] = dfb.apply(
        lambda r: (r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
                   r.get("amp"), r.get("batch_size"), r.get("seed")), axis=1)

    # Para cada clave, escoge el run baseline con menor MAE y usa sus dos métricas
    base_rows = []
    for key, g in dfb[dfb["method_base"].astype(str).str.lower() == baseline.lower()].groupby("__key__"):
        if g.empty: 
            continue
        idx = g["circuito2_final_mae"].astype(float).idxmin()
        row = g.loc[idx]
        base_rows.append({
            "__key__": key,
            "baseline_mae": float(row["circuito2_final_mae"]),
            "baseline_forget": float(row["avg_forget_rel"])
        })
    base_tbl = pd.DataFrame(base_rows)

    out = dfb.merge(base_tbl, on="__key__", how="left")
    out["delta_mae_vs_base"]    = out["circuito2_final_mae"] - out["baseline_mae"]
    out["delta_forget_vs_base"] = out["avg_forget_rel"]      - out["baseline_forget"]
    return out


if RELATIVE_BASELINE:
    df_rel = compute_relative_to_baseline(df_sel, baseline=RELATIVE_BASELINE, strict=BASELINE_MATCH_STRICT)
    if df_rel is not None:
        out_csv = SUMMARY / f"relative_to_{RELATIVE_BASELINE}.csv"
        df_rel.to_csv(out_csv, index=False)
        print(f"[OK] Relative-to-baseline → {out_csv}")
        # Top-10 mejoras vs baseline (orden: delta_mae luego delta_forget)
        mask_has_base = (~df_rel["baseline_mae"].isna()) & (~df_rel["baseline_forget"].isna())
        top_improve = (df_rel[mask_has_base]
                       .sort_values(["delta_mae_vs_base","delta_forget_vs_base"], ascending=[True, True])
                       .head(10))
        cols_show = ["run_dir","method","method_base","circuito2_final_mae","avg_forget_rel",
                     "baseline_mae","baseline_forget","delta_mae_vs_base","delta_forget_vs_base"]
        display(top_improve[cols_show])


[OK] Relative-to-baseline → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/relative_to_ewc.csv


,run_dir,method,method_base,circuito2_final_mae,avg_forget_rel,baseline_mae,baseline_forget,delta_mae_vs_base,delta_forget_vs_base
16,continual_fast_sa-snn_fast3_sa_k0p10_tau28_vt1...,sa-snn,sa-snn,0.157522,0.157920,0.195218,0.40501,-0.037696,-0.247090
31,continual_fast_sca-snn_bins32_beta0.8_bias0.08...,sca-snn_bins32_beta0.8_bias0.08_temp0.12_ab12_...,sca-snn,0.160026,0.120142,0.195218,0.40501,-0.035192,-0.284868
30,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn_bins32_beta0.8_bias0.12_temp0.16_ab12_...,sca-snn,0.161127,0.218797,0.195218,0.40501,-0.034090,-0.186214
55,continual_fast_as-snn_gr_0.55_lam_0.4_att_f6_f...,as-snn_gr_0.55_lam_0.4_att_f6,as-snn,0.163275,0.095513,0.195218,0.40501,-0.031943,-0.309497
25,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn_bins32_beta0.8_bias0.12_temp0.12_ab12_...,sca-snn,0.164060,0.173845,0.195218,0.40501,-0.031158,-0.231166
40,continual_fast_sca-snn_bins64_beta0.8_bias0.08...,sca-snn_bins64_beta0.8_bias0.08_temp0.12_ab12_...,sca-snn,0.164407,0.142466,0.195218,0.40501,-0.030811,-0.262545
28,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn_bins32_beta0.8_bias0.12_temp0.1_ab12_f...,sca-snn,0.164804,0.297443,0.195218,0.40501,-0.030413,-0.107567
36,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn_bins32_beta0.8_bias0.12_temp0.12_ab12_...,sca-snn,0.165590,0.102540,0.195218,0.40501,-0.029628,-0.302470
39,continual_fast_sca-snn_bins32_beta0.8_bias0.08...,sca-snn_bins32_beta0.8_bias0.08_temp0.1_ab12_f...,sca-snn,0.165673,0.137776,0.195218,0.40501,-0.029545,-0.267234
1,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,as-snn_gr_0.6_lam_0.4_att_f6,as-snn,0.165940,0.187910,0.195218,0.40501,-0.029278,-0.217101


In [8]:
# Celda 6 — Top-N global y Pareto (versión robusta)

# helpers (por si no vienes de la Celda 5)
try:
    _ = _ensure_mae_columns
except NameError:
    def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        has_c2 = "c2_final_mae" in df.columns
        has_circ2 = "circuito2_final_mae" in df.columns
        if has_c2 and not has_circ2:
            df["circuito2_final_mae"] = df["c2_final_mae"]
        elif has_circ2 and not has_c2:
            df["c2_final_mae"] = df["circuito2_final_mae"]
        return df

df_rank = _ensure_mae_columns(df_report if "df_report" in globals() else df_sel.copy())
if "score_eff" not in df_rank.columns:
    df_rank = df_rank.copy()
    df_rank["score_eff"] = np.nan
if "method_base" not in df_rank.columns:
    df_rank["method_base"] = df_rank["method"] if "method" in df_rank.columns else "unknown"

MAE_COL = "circuito2_final_mae" if "circuito2_final_mae" in df_rank.columns else ("c2_final_mae" if "c2_final_mae" in df_rank.columns else "circuito2_final_mae")

# Top-N compuesto
TOPN = 6
if df_rank.empty:
    topn = df_rank.copy()
    print("[WARN] df_rank está vacío; Top-N vacío.")
else:
    cols_sort = [c for c in ["score_eff", MAE_COL] if c in df_rank.columns]
    if not cols_sort:
        topn = df_rank.head(TOPN)
    else:
        topn = df_rank.sort_values(cols_sort, ascending=[True] * len(cols_sort)).head(TOPN)
out_csv = SUMMARY / "top6_composite.csv"
topn.to_csv(out_csv, index=False)
print("[OK] Top-6 →", out_csv)
cols_show = [c for c in ["run_dir","method_base","seed",MAE_COL,"avg_forget_rel","emissions_kg","score_eff"] if c in topn.columns]
display(topn[cols_show] if cols_show else topn)

# Pareto (minimiza MAE y olvido); NaN -> +inf para no dominar
tmp = _ensure_mae_columns(df_report.copy())
if "method_base" not in tmp.columns:
    tmp["method_base"] = tmp["method"] if "method" in tmp.columns else "unknown"
if MAE_COL not in tmp.columns:
    tmp[MAE_COL] = np.nan

tmp["_olvido_for_pareto"] = pd.to_numeric(tmp.get("avg_forget_rel", np.nan), errors="coerce")
tmp.loc[tmp["_olvido_for_pareto"].isna(), "_olvido_for_pareto"] = np.inf
tmp["_mae_for_pareto"] = pd.to_numeric(tmp.get(MAE_COL, np.nan), errors="coerce")
tmp.loc[tmp["_mae_for_pareto"].isna(), "_mae_for_pareto"] = np.inf

pareto_idx = []
vals = tmp[["_mae_for_pareto", "_olvido_for_pareto"]].values
for i in range(len(tmp)):
    mae_i, forg_i = vals[i]
    dominated = False
    for j in range(len(tmp)):
        if j == i: continue
        mae_j, forg_j = vals[j]
        if (mae_j <= mae_i) and (forg_j <= forg_i) and ((mae_j < mae_i) or (forg_j < forg_i)):
            dominated = True
            break
    if not dominated:
        pareto_idx.append(i)

df_pareto = tmp.iloc[pareto_idx].drop(columns=["_olvido_for_pareto", "_mae_for_pareto"])
out_csv = SUMMARY / "pareto.csv"
df_pareto.to_csv(out_csv, index=False)
print("[OK] Pareto →", out_csv)
cols_show = [c for c in ["run_dir","method_base",MAE_COL,"avg_forget_rel"] if c in df_pareto.columns]
display(df_pareto[cols_show] if cols_show else df_pareto)


[OK] Top-6 → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/top6_composite.csv


,run_dir,method_base,seed,circuito2_final_mae,avg_forget_rel,emissions_kg,score_eff
65,continual_fast_sa-snn_fast3_sa_k0p10_tau28_vt1...,sa-snn,42.0,0.157522,0.157920,0.005827,NaN
108,continual_fast_sca-snn_bins32_beta0.8_bias0.08...,sca-snn,42.0,0.160026,0.120142,0.004915,NaN
106,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn,42.0,0.161127,0.218797,0.005200,NaN
210,continual_fast_as-snn_gr_0.55_lam_0.4_att_f6_f...,as-snn,42.0,0.163275,0.095513,0.005170,NaN
91,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn,42.0,0.164060,0.173845,0.005105,NaN
147,continual_fast_sca-snn_bins64_beta0.8_bias0.08...,sca-snn,42.0,0.164407,0.142466,0.004921,NaN


[OK] Pareto → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/pareto.csv


,run_dir,method_base,circuito2_final_mae,avg_forget_rel
48,continual_fast_sca-snn_bins32_beta0.8_bias0.12...,sca-snn,0.268958,0.000000
65,continual_fast_sa-snn_fast3_sa_k0p10_tau28_vt1...,sa-snn,0.157522,0.157920
94,continual_fast_sa-snn_fast3_sa_k0p10_tau32_vt1...,sa-snn,0.184305,0.091378
108,continual_fast_sca-snn_bins32_beta0.8_bias0.08...,sca-snn,0.160026,0.120142
185,continual_fast_as-snn_gr_0.6_lam_0.35_att_f6_f...,as-snn,0.209745,0.001521
210,continual_fast_as-snn_gr_0.55_lam_0.4_att_f6_f...,as-snn,0.163275,0.095513
218,continual_fast_ewc_lam_4e+08_fast3_ewc_l35e8_f...,ewc,0.204162,0.065061


In [9]:
# Celda 7 — Scatter MAE vs Olvido

import matplotlib.pyplot as plt

def _scatter(df, title, out_png: Path):
    x = pd.to_numeric(df["avg_forget_rel"], errors="coerce")
    y = pd.to_numeric(df["circuito2_final_mae"], errors="coerce")
    plt.figure(figsize=(7,5))
    plt.scatter(x, y)  # sin estilos ni colores específicos
    plt.xlabel("avg_forget_rel (↓ mejor)")
    plt.ylabel("circuito2_final_mae (↓ mejor)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()

_scatter(df_sel, "Todos", SUMMARY / "scatter_all.png")
_scatter(winners if "winners" in globals() else df_sel.head(0), "Winners por método", SUMMARY / "scatter_winners.png")
print("[OK] Scatter →", SUMMARY / "scatter_all.png")
print("[OK] Scatter →", SUMMARY / "scatter_winners.png")


[OK] Scatter → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/scatter_all.png
[OK] Scatter → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/scatter_winners.png


In [10]:
# Celda 8 — Trazabilidad por run

def explain_sources(run_row):
    rd = _abs_run_dir(run_row["run_dir"])
    exists = {
        "per_task_perf.json": (rd / "per_task_perf.json").exists(),
        "per_task_perf.csv":  (rd / "per_task_perf.csv").exists(),
        "forgetting.json":    (rd / "forgetting.json").exists(),
        "eval_matrix.csv":    (rd / "eval_matrix.csv").exists(),
        "eval_matrix.json":   (rd / "eval_matrix.json").exists(),
        "efficiency_summary.json": (rd / "efficiency_summary.json").exists(),
        "emissions.csv":      (rd / "emissions.csv").exists(),
        "run_row.json":       (rd / "run_row.json").exists(),
        "task_1_circuito1/manifest.json": (rd / "task_1_circuito1/manifest.json").exists(),
    }
    return exists

print("## NaN en selección ##")
cols_check = ["circuito1_best_mae","circuito1_final_mae","circuito2_final_mae","avg_forget_rel"]
mask_nans = df_sel[cols_check].isna().any(axis=1)
df_nans = df_sel[mask_nans].copy()
print(f"[INFO] {len(df_nans)} runs con NaN en {cols_check}:")
display(df_nans[["run_dir","method_base","preset","encoder","seed","T","amp","batch_size"] + cols_check] if len(df_nans) else df_nans)

for _, r in df_sel.iterrows():
    ex = explain_sources(r)
    ok_flags = " | ".join([f"{k} → {'OK' if v else 'NO'}" for k,v in ex.items()])
    print(f"— {r['run_dir']}\n    {ok_flags}")


## NaN en selección ##
[INFO] 0 runs con NaN en ['circuito1_best_mae', 'circuito1_final_mae', 'circuito2_final_mae', 'avg_forget_rel']:


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base,batch_size_filled


— continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_fast3_as_gr060_lam036_ema090_l1_att_f6_rate_model-PilotNetSNN_66x200_gray_seed_42
    per_task_perf.json → OK | per_task_perf.csv → OK | forgetting.json → OK | eval_matrix.csv → OK | eval_matrix.json → OK | efficiency_summary.json → OK | emissions.csv → OK | run_row.json → OK | task_1_circuito1/manifest.json → OK
— continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fast3_as_gr060_lam040_ema095_att_f6_l2_rate_model-PilotNetSNN_66x200_gray_seed_42
    per_task_perf.json → OK | per_task_perf.csv → OK | forgetting.json → OK | eval_matrix.csv → OK | eval_matrix.json → OK | efficiency_summary.json → OK | emissions.csv → OK | run_row.json → OK | task_1_circuito1/manifest.json → OK
— continual_fast_sca-snn_bins32_beta0.8_bias0.1_temp0.1_ab12_flat1_fast3_sca_bias010_t010_ab12_taf37_hd097_f6_rate_model-PilotNetSNN_66x200_gray_seed_42
    per_task_perf.json → OK | per_task_perf.csv → OK | forgetting.json → OK | eval_matrix.csv → OK | eval_matrix.json → OK |

In [11]:
# Celda 9 — Sanidad naive vs CL

DF = df_sel.copy()
agg = (
    DF.groupby("method_base", dropna=False)
      .agg(forget_median=("avg_forget_rel","median"),
           forget_mean=("avg_forget_rel","mean"),
           mae_mean=("circuito2_final_mae","mean"),
           runs=("run_dir","count"))
      .reset_index()
)
display(agg)

naive_med = agg.loc[agg["method_base"]=="naive","forget_median"].values
cl_med    = agg.loc[agg["method_base"]!="naive","forget_median"].median()
print(f"[CHECK] naive_median_forget={naive_med[0] if len(naive_med) else np.nan} vs CL_median={cl_med}")
if len(naive_med) and (np.isnan(naive_med[0]) or (not np.isnan(cl_med) and naive_med[0] <= cl_med)):
    print("[WARN] Naive NO olvida más que CL. Revisa eval_matrix/forgetting.json o implementación.")



,method_base,forget_median,forget_mean,mae_mean,runs
0,as-snn,0.189045,0.203202,0.186956,15
1,ewc,0.360302,0.343274,0.209570,12
2,sa-snn,0.168281,0.210636,0.170349,5
3,sca-snn,0.195425,0.184188,0.174166,30


[CHECK] naive_median_forget=nan vs CL_median=0.19223519914211898


In [12]:
# Celda 9b — Sanity-check EWC (parse logs en carpetas de run)

import glob

def _find_log_files(run_dir: Path):
    pats = ["*.log", "stdout*.txt", "train*.log"]
    files = []
    for p in pats:
        files += list(run_dir.glob(p))
        files += list((run_dir / "task_1_circuito1").glob(p))
        files += list((run_dir / "task_2_circuito2").glob(p))
    return files

def _parse_ewc_lines(text: str):
    # Busca líneas del estilo: [EWC] base=... | pen=... | pen/base=...
    bases, pens, ratios = [], [], []
    for line in text.splitlines():
        if "[EWC]" in line:
            # intenta extraer números
            try:
                # formato flexible
                # ej: [EWC] base=0.007436 | pen=0 | pen/base=0.000
                m_base = re.search(r"base\s*=\s*([0-9.eE+-]+)", line)
                m_pen  = re.search(r"pen\s*=\s*([0-9.eE+-]+)", line)
                m_rat  = re.search(r"pen/base\s*=\s*([0-9.eE+-]+)", line)
                if m_base: bases.append(float(m_base.group(1)))
                if m_pen:  pens.append(float(m_pen.group(1)))
                if m_rat:  ratios.append(float(m_rat.group(1)))
            except Exception:
                pass
    return bases, pens, ratios

rows = []
for _, r in df_sel.iterrows():
    if not str(r.get("method","")).lower().startswith("ewc") and "ewc" not in str(r.get("method","")).lower():
        continue
    rd = _abs_run_dir(r["run_dir"])
    logs = _find_log_files(rd)
    all_bases, all_pens, all_ratios = [], [], []
    for lf in logs:
        try:
            txt = lf.read_text(encoding="utf-8", errors="ignore")
            b, p, q = _parse_ewc_lines(txt)
            all_bases += b; all_pens += p; all_ratios += q
        except Exception:
            pass
    rows.append({
        "run_dir": r["run_dir"],
        "has_logs": len(logs) > 0,
        "pen_gt0_count": int(sum(1 for x in all_pens if (isinstance(x,float) and x > 0))),
        "pen_entries": len(all_pens),
        "ratio_gt0_count": int(sum(1 for x in all_ratios if (isinstance(x,float) and x > 0))),
        "ratio_entries": len(all_ratios),
    })

df_ewc_sanity = pd.DataFrame(rows)
out_csv = SUMMARY / "ewc_sanity.csv"
df_ewc_sanity.to_csv(out_csv, index=False)
print("[OK] EWC sanity →", out_csv)
display(df_ewc_sanity)


[OK] EWC sanity → /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06/ewc_sanity.csv


,run_dir,has_logs,pen_gt0_count,pen_entries,ratio_gt0_count,ratio_entries
0,continual_fast_ewc_lam_6e+08_fast3_ewc_l60e8_f...,False,0,0,0,0
1,continual_fast_ewc_lam_4e+08_fast3_ewc_l35e8_f...,False,0,0,0,0
2,continual_fast_ewc_lam_5e+08_fast3_ewc_l50e8_f...,False,0,0,0,0
3,continual_fast_ewc_lam_3e+08_fast3_ewc_l30e8_f...,False,0,0,0,0
4,continual_fast_ewc_lam_2e+09_fast3_ewc_l20e9_f...,False,0,0,0,0
5,continual_fast_ewc_lam_3e+09_fast3_ewc_l30e9_f...,False,0,0,0,0
6,continual_fast_ewc_lam_8e+08_fast3_ewc_l8e8_fb...,False,0,0,0,0
7,continual_fast_ewc_lam_4e+08_fast3_ewc_l40e8_f...,False,0,0,0,0
8,continual_fast_ewc_lam_4e+08_fast3_ewc_l35e8_f...,False,0,0,0,0
9,continual_fast_ewc_lam_2e+08_fast3_ewc_l25e8_f...,False,0,0,0,0


In [13]:
# Celda 10 — Export final
print("[OK] Artifacts en:", SUMMARY)
for p in sorted(SUMMARY.glob("*.csv")) + sorted(SUMMARY.glob("*.png")):
    print(" -", p.name)


[OK] Artifacts en: /home/cesar/proyectos/TFM_SNN/outputs/summary/paper_set_accurate_2025-11-06
 - ewc_sanity.csv
 - pareto.csv
 - relative_to_ewc.csv
 - relative_to_naive.csv
 - results_table_fromdisk.csv
 - selection_table.csv
 - top6_composite.csv
 - winners_per_method.csv
 - winners_per_methodbase.csv
 - scatter_all.png
 - scatter_winners.png
